In [18]:
import numpy as np
import pandas as pd
import dill as dl
import matplotlib.pyplot as plt
import sys
import pickle
import itertools
import re
import os
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as sm_f

In [19]:
with open("D:/DZ/Course_6/Diploma/Data/two_time_points.csv", "rb") as inp:
    two_time_points = pickle.load(inp)

In [20]:
with open("D:/DZ/Course_6/Diploma/Data/financial_data.csv", "rb") as inp:
    temp_fin = pickle.load(inp)

In [21]:
with open("D:/DZ/Course_5/Курсовая/data/data_for_models.csv", "rb") as inp:
    data_for_models = pickle.load(inp)

In [22]:
with open("D:/DZ/Course_6/Diploma/Data/full_ind_data.csv", "rb") as inp:
    full_ind_data = pickle.load(inp)

In [23]:
pd.set_option("max_columns", None)

In [24]:
two_time_points

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,treatment_period
0,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0
1,Владивостокский,Приморский край,1,1,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,1
2,Хасанский район,Приморский край,0,1,2016,NaN,0.0,NaN,4.20,32139.0,NaN,0.023735,0.011735,NaN,0
3,Хасанский район,Приморский край,0,1,2021,NaN,NaN,NaN,6.20,30318.0,NaN,0.020031,0.004033,0.002294,1
4,Артемовский,Приморский край,1,1,2016,NaN,0.0,NaN,2.62,114873.0,NaN,0.009542,0.009249,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
303,Мошенской район,Новгородская область,0,0,2021,45.00,NaN,NaN,3.32,5797.0,NaN,NaN,0.007950,NaN,1
304,Пестовский район,Новгородская область,0,1,2016,88.77,0.0,NaN,0.63,20934.0,NaN,NaN,0.003777,NaN,0
305,Пестовский район,Новгородская область,0,1,2021,75.10,NaN,NaN,0.82,19579.0,NaN,NaN,0.009602,0.014286,1
306,Хвойнинский район,Новгородская область,0,1,2016,27.61,10.0,NaN,24.65,14950.0,NaN,NaN,0.003137,0.012089,0


In [25]:
temp_fin.head()

,region,raion,sub_indicator,year,indicator,times,values,budget_type,is_amendment,amendment_date,population
0,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2013,расходы бюджета,Not present,1703.560574,actual_spending,NaN,NaN,271445.0
1,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2014,расходы бюджета,Not present,1757.246786,actual_spending,NaN,NaN,273382.0
2,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2015,расходы бюджета,Not present,1315.128876,actual_spending,NaN,NaN,276090.0
3,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2016,расходы бюджета,Not present,2508.275869,actual_spending,NaN,NaN,276691.0
4,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2017,расходы бюджета,Not present,3008.628778,actual_spending,NaN,NaN,277648.0


In [26]:
temp_fin['values'].min()

-46465.5756015087

In [27]:
full_ind_data.head()

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed
0,Владивостокский,Приморский край,1,1,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211
1,Владивостокский,Приморский край,1,1,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417
2,Владивостокский,Приморский край,1,1,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201
3,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074
4,Владивостокский,Приморский край,1,1,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201


In [28]:
mapping = {"Приморский край": 2016, "Мурманская область": 2016, "Костромская область": 2015, "Республика Алтай": 2014, "Республика Марий Эл": 2014, "Хабаровский край": 2014, 
           "Тульская область": 2014, "Новгородская область": 2016}

# Goods

In [29]:
full_ind_data.year.max()

2021

In [30]:
for i in range(-8,4):
    if i < 0:
        string = f"minus_{abs(i)}"
    else:
        string = f'plus_{i}'
    full_ind_data[string] = 0
    for region in mapping.keys():
        full_ind_data.loc[(full_ind_data.region == region) & (full_ind_data.year == mapping[region]+5+i) & (full_ind_data.treatment_1 == 1), string] = 1

In [31]:
full_ind_data.head(10)

,raion,region,treatment_1,treatment_2,year,bad_roads,bad_schools,unfinished_construction,housing,population,lack_transportation,heat_grid_fixed,water_grid_fixed,sewage_grid_fixed,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
0,Владивостокский,Приморский край,1,1,2013,NaN,NaN,3400.00,4.05,625868.0,NaN,0.050240,0.004470,0.001211,1,0,0,0,0,0,0,0,0,0,0,0
1,Владивостокский,Приморский край,1,1,2014,NaN,NaN,3400.00,3.88,630027.0,NaN,0.020670,0.008552,0.004417,0,1,0,0,0,0,0,0,0,0,0,0
2,Владивостокский,Приморский край,1,1,2015,NaN,0.0,21529.93,5.00,631387.0,NaN,0.019753,0.009448,0.003201,0,0,1,0,0,0,0,0,0,0,0,0
3,Владивостокский,Приморский край,1,1,2016,NaN,0.0,2411.41,5.17,633167.0,NaN,0.011566,0.006601,0.001074,0,0,0,1,0,0,0,0,0,0,0,0
4,Владивостокский,Приморский край,1,1,2017,NaN,0.0,NaN,3.19,633414.0,NaN,0.013158,0.003128,0.003201,0,0,0,0,1,0,0,0,0,0,0,0
5,Владивостокский,Приморский край,1,1,2018,NaN,0.0,34023.34,4.21,633102.0,NaN,0.016002,0.005096,0.002653,0,0,0,0,0,1,0,0,0,0,0,0
6,Владивостокский,Приморский край,1,1,2019,NaN,0.0,67570.22,4.48,633144.0,NaN,0.028937,0.013167,0.002075,0,0,0,0,0,0,1,0,0,0,0,0
7,Владивостокский,Приморский край,1,1,2020,NaN,NaN,106191.72,4.80,634665.0,NaN,0.025499,0.007071,0.001982,0,0,0,0,0,0,0,1,0,0,0,0
8,Владивостокский,Приморский край,1,1,2021,NaN,NaN,NaN,4.74,628623.0,NaN,0.020431,0.010109,0.003427,0,0,0,0,0,0,0,0,1,0,0,0
9,Хасанский район,Приморский край,0,1,2013,NaN,6.3,116999.00,4.30,34112.0,NaN,0.028997,0.009722,0.008197,0,0,0,0,0,0,0,0,0,0,0,0


In [32]:
treat_1_roads = full_ind_data[["raion", "region", "bad_roads", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_roads.dropna(inplace=True)

roads = sm_f.ols(formula="bad_roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", 
                 data=treat_1_roads).fit()
roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              bad_roads   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.844
Method:                 Least Squares   F-statistic:                     39.31
Date:                Fri, 12 May 2023   Prob (F-statistic):          2.17e-280
Time:                        23:01:05   Log-Likelihood:                -3510.8
No. Observations:                 955   AIC:                             7294.
Df Residuals:                     819   BIC:                             7955.
Df Model:                         135                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                           11.0381      3.854      2.864      0.004       3.473      18.604
C(raion)[T.Антроповский район]                                      75.2213      5.017     14.994      0.000      65.374      85.069
C(raion)[T.Аяно-Майский район]                                      88.9619     11.017      8.075      0.000      67.338     110.586
C(raion)[T.Батецкий район]                                          51.8169      5.017     10.329      0.000      41.970      61.664
C(raion)[T.Бикинский район]                                         21.2425      5.160      4.117      0.000      11.114      31.371
C(raion)[T.Боровичский район]                                       26.8911      5.454      4.931      0.000      16.186      37.596
C(raion)[T.Буйский район]                                           30.3380      5.017      6.047      0.000      20.491      40.185
C(raion)[T.Валдайский район]                                        27.7535      5.017      5.532      0.000      17.906      37.601
C(raion)[T.Ванинский район]                                         13.0313      5.160      2.525      0.012       2.902      23.160
C(raion)[T.Великий Новгород]                                        22.4056      5.454      4.108      0.000      11.700      33.111
C(raion)[T.Верхнебуреинский район]                                   9.4684      5.891      1.607      0.108      -2.096      21.033
C(raion)[T.Волжский район]                                          66.2319      5.432     12.193      0.000      55.570      76.894
C(raion)[T.Волотовский район]                                       20.6161      5.164      3.992      0.000      10.479      30.753
C(raion)[T.Вохомский район]                                         75.3880      5.017     15.027      0.000      65.541      85.235
C(raion)[T.Вяземский район]                                         -4.3365      5.017     -0.864      0.388     -14.184       5.511
C(raion)[T.Галичский район]                                         74.2480      5.017     14.800      0.000      64.401      84.095
C(raion)[T.Горномарийский район]                                    81.2591      5.017     16.198      0.000      71.412      91.106
C(raion)[T.Город Волжск]                                            19.4602      5.017      3.879      0.000       9.613      29.307
C(raion)[T.Город Горно-Алтайск]                                     12.9919      5.432      2.392      0.017       2.330      23.654
C(raion)[T.Город Козьмодемьянск]                                    55.5913      6.331      8.781      0.000      43.165      68.017
C(raion)[T.Город Комсомо

In [33]:
treat_1_roads.year.unique()

array([2015, 2016, 2017, 2018, 2019, 2020, 2013, 2014, 2021], dtype=int64)

In [34]:
roads.params[(len(roads.params)-10):(len(roads.params))].values

array([ 5.47719537,  9.92386034,  6.4338138 ,  4.0949312 , -0.22755052,
        0.92273629,  2.29486656,  3.08192109,  2.83932429,  5.39511047])

In [35]:
treat_1_roads.head()

,raion,region,bad_roads,year,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
281,Печенгский район,Мурманская область,9.00,2015,0,0,0,0,0,0,0,0,0,0,0,0
282,Печенгский район,Мурманская область,6.26,2016,0,0,0,0,0,0,0,0,0,0,0,0
283,Печенгский район,Мурманская область,7.86,2017,0,0,0,0,0,0,0,0,0,0,0,0
284,Печенгский район,Мурманская область,11.72,2018,0,0,0,0,0,0,0,0,0,0,0,0
285,Печенгский район,Мурманская область,24.80,2019,0,0,0,0,0,0,0,0,0,0,0,0


In [36]:
treat_1_schools = full_ind_data[["raion", "region", "bad_schools", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2","minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_schools.dropna(inplace=True)

schools = sm_f.ols(formula="bad_schools ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_schools).fit()
schools.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            bad_schools   R-squared:                       0.447
Model:                            OLS   Adj. R-squared:                  0.308
Method:                 Least Squares   F-statistic:                     3.215
Date:                Fri, 12 May 2023   Prob (F-statistic):           1.22e-20
Time:                        23:01:12   Log-Likelihood:                -2416.6
No. Observations:                 638   AIC:                             5091.
Df Residuals:                     509   BIC:                             5666.
Df Model:                         128                                         
Covariance Type:            nonrobust                                         
========================================================================================================
                                           coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------------------
Intercept                               51.6572      8.649      5.973      0.000      34.666      68.649
C(raion)[T.Анучинский район]           -51.8705     10.037     -5.168      0.000     -71.590     -32.151
C(raion)[T.Арсеньевский]               -51.8705     10.037     -5.168      0.000     -71.590     -32.151
C(raion)[T.Артемовский]                -53.4791     10.217     -5.234      0.000     -73.551     -33.407
C(raion)[T.Аяно-Майский район]         -25.4428     14.681     -1.733      0.084     -54.285       3.399
C(raion)[T.Батецкий район]             -44.8316      9.797     -4.576      0.000     -64.079     -25.585
C(raion)[T.Бикинский район]            -41.4693     12.005     -3.454      0.001     -65.055     -17.884
C(raion)[T.Боровичский район]          -49.8713     10.454     -4.771      0.000     -70.409     -29.334
C(raion)[T.Валдайский район]           -51.4983      9.797     -5.257      0.000     -70.745     -32.251
C(raion)[T.Ванинский район]            -44.7958     14.765     -3.034      0.003     -73.805     -15.787
C(raion)[T.Великий Новгород]           -45.5540      9.900     -4.601      0.000     -65.004     -26.104
C(raion)[T.Владивостокский]            -49.9617     10.441     -4.785      0.000     -70.475     -29.449
C(raion)[T.Волжский район]             -47.2334      9.879     -4.781      0.000     -66.642     -27.825
C(raion)[T.Волотовский район]          -34.8316      9.797     -3.555      0.000     -54.079     -15.585
C(raion)[T.Горномарийский район]       -50.0865      9.624     -5.204      0.000     -68.994     -31.179
C(raion)[T.Город Волжск]               -52.3003      9.499     -5.506      0.000     -70.962     -33.639
C(raion)[T.Город Горно-Алтайск]        -45.7334      9.879     -4.629      0.000     -65.142     -26.325
C(raion)[T.Город Йошкар-Ола]           -45.3118      9.955     -4.551      0.000     -64.871     -25.753
C(raion)[T.Город Козьмодемьянск]       -51.6722      9.624     -5.369      0.000     -70.579     -32.765
C(raion)[T.Город Новомосковск]         -50.8360     10.384     -4.896      0.000     -71.236     -30.436
C(raion)[T.Город Тула]                 -46.8187     10.590     -4.421      0.000     -67.623     -26.014
C(raion)[T.Дальнегорский]              -49.1905     10.037     -4.901      0.000     -68.910     -29.471
C(raion)[T.Дальнереченский]            -47.1017     10.441     -4.511      0.000     -67.615     -26.589
C(raion)[T.Демянский район]            -38.9983      9.797     -3.981      0.000     -58.245     -19.751
C(raion)[T.ЗАТО Фокино]                -53.1174     10.992     -4.832      0.000     -74.713     -31.522
C(raion)[T.Заокский район]             -13.1587     10.590     -1.243      0.215     -33.963       7.646
C(raion)[T.Звениговский район]        

In [37]:
treat_1_sewerage =  full_ind_data[["raion", "region", "year", "sewage_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2","minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_sewerage.dropna(inplace=True)

sewerage = sm_f.ols(formula="sewage_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_sewerage).fit()
sewerage.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:      sewage_grid_fixed   R-squared:                       0.635
Model:                            OLS   Adj. R-squared:                  0.518
Method:                 Least Squares   F-statistic:                     5.431
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.76e-36
Time:                        23:01:19   Log-Likelihood:                 1044.0
No. Observations:                 491   AIC:                            -1848.
Df Residuals:                     371   BIC:                            -1344.
Df Model:                         119                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                            0.0065      0.016      0.416      0.678      -0.024       0.037
C(raion)[T.Анучинский район]                                        -0.0081      0.037     -0.219      0.827      -0.080       0.064
C(raion)[T.Арсеньевский]                                            -0.0098      0.024     -0.402      0.688      -0.058       0.038
C(raion)[T.Артемовский]                                             -0.0097      0.021     -0.462      0.644      -0.051       0.032
C(raion)[T.Бикинский район]                                          0.0028      0.028      0.099      0.922      -0.052       0.058
C(raion)[T.Боровичский район]                                       -0.0080      0.021     -0.380      0.704      -0.050       0.033
C(raion)[T.Буйский район]                                           -0.0051      0.028     -0.181      0.856      -0.060       0.050
C(raion)[T.Валдайский район]                                         0.0039      0.019      0.212      0.833      -0.033       0.040
C(raion)[T.Ванинский район]                                         -0.0072      0.020     -0.371      0.711      -0.046       0.031
C(raion)[T.Великий Новгород]                                        -0.0061      0.021     -0.290      0.772      -0.048       0.035
C(raion)[T.Верхнебуреинский район]                                   0.0083      0.028      0.297      0.766      -0.047       0.063
C(raion)[T.Владивостокский]                                         -0.0101      0.020     -0.496      0.620      -0.050       0.030
C(raion)[T.Вяземский район]                                         -0.0067      0.022     -0.298      0.766      -0.051       0.037
C(raion)[T.Галичский район]                                          0.0345      0.037      0.941      0.347      -0.038       0.107
C(raion)[T.Город Горно-Алтайск]                                      0.0092      0.039      0.239      0.811      -0.067       0.085
C(raion)[T.Город Комсомольск-на-Амуре]                              -0.0043      0.022     -0.199      0.842      -0.047       0.038
C(raion)[T.Город Новомосковск]                                      -0.0068      0.029     -0.238      0.812      -0.063       0.049
C(raion)[T.Город Тула]                                              -0.0082      0.023     -0.350      0.726      -0.054       0.038
C(raion)[T.Город Хабаровск]                                          0.0018      0.020      0.088      0.930      -0.038       0.042
C(raion)[T.Дальнегорский]                                           -0.0096      0.020     -0.490      0.624      -0.048       0.029
C(raion)[T.Дальнереченск

In [38]:
treat_1_heat = full_ind_data[["raion", "region", "year", "heat_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2","minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_heat.dropna(inplace=True)

heat = sm_f.ols(formula="heat_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_heat).fit() 
heat.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        heat_grid_fixed   R-squared:                       0.318
Model:                            OLS   Adj. R-squared:                  0.170
Method:                 Least Squares   F-statistic:                     2.155
Date:                Fri, 12 May 2023   Prob (F-statistic):           0.000133
Time:                        23:01:19   Log-Likelihood:                 628.58
No. Observations:                 260   AIC:                            -1163.
Df Residuals:                     213   BIC:                            -995.8
Df Model:                          46                                         
Covariance Type:            nonrobust                                         
=====================================================================================================
                                        coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------------
Intercept                             0.0221      0.012      1.891      0.060      -0.001       0.045
C(raion)[T.Арсеньевский]             -0.0023      0.013     -0.175      0.862      -0.029       0.024
C(raion)[T.Артемовский]              -0.0076      0.016     -0.483      0.629      -0.039       0.023
C(raion)[T.Владивостокский]           0.0048      0.016      0.307      0.759      -0.026       0.036
C(raion)[T.Дальнегорский]            -0.0084      0.013     -0.625      0.532      -0.035       0.018
C(raion)[T.Дальнереченский]           0.0104      0.016      0.656      0.512      -0.021       0.041
C(raion)[T.ЗАТО Фокино]               0.0527      0.015      3.619      0.000       0.024       0.081
C(raion)[T.Кавалеровский район]      -0.0008      0.016     -0.049      0.961      -0.032       0.030
C(raion)[T.Кировский район]           0.0078      0.013      0.585      0.559      -0.019       0.034
C(raion)[T.Красноармейский район]     0.0263      0.016      1.666      0.097      -0.005       0.057
C(raion)[T.Лазовский район]           0.0364      0.014      2.664      0.008       0.009       0.063
C(raion)[T.Лесозаводский]             0.0136      0.013      1.017      0.310      -0.013       0.040
C(raion)[T.Михайловский район]        0.0090      0.013      0.671      0.503      -0.017       0.035
C(raion)[T.Надеждинский район]        0.0245      0.013      1.826      0.069      -0.002       0.051
C(raion)[T.Находкинский]              0.0107      0.016      0.677      0.499      -0.020       0.042
C(raion)[T.Октябрьский район]         0.0124      0.014      0.905      0.367      -0.015       0.039
C(raion)[T.Ольгинский район]          0.0037      0.013      0.278      0.781      -0.023       0.030
C(raion)[T.Партизанский]              0.0003      0.013      0.026      0.979      -0.026       0.027
C(raion)[T.Пограничный район]         0.0338      0.014      2.334      0.021       0.005       0.062
C(raion)[T.Пожарский район]          -0.0116      0.013     -0.866      0.387      -0.038       0.015
C(raion)[T.Спасск-Дальний]            0.0032      0.016      0.204      0.838      -0.028       0.034
C(raion)[T.Спасский район]           -0.0009      0.016     -0.057      0.955      -0.032       0.030
C(raion)[T.Тернейский район]         -0.0081      0.014     -0.592      0.555      -0.035       0.019
C(raion)[T.Уссурийский]               0.0187      0.016      1.183      0.238      -0.012       0.050
C(raion)[T.Ханкайский район]         -0.0014      0.014     -0.101      0.920      -0.028       0.026
C(raion)[T.Хасанский район]           0.0097      0.013      0.727      0.468      -0.017       0.036
C(raion)[T.Хорольский район]          0.0163      0.014      1.195      0.233      -0.011       0.043
C(raion)[T.Черниговский

In [39]:
treat_1_water = full_ind_data[["raion", "region", "year", "water_grid_fixed", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2","minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
treat_1_water.dropna(inplace=True)

water = sm_f.ols(formula="water_grid_fixed ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=treat_1_water).fit()
water.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       water_grid_fixed   R-squared:                       0.365
Model:                            OLS   Adj. R-squared:                  0.235
Method:                 Least Squares   F-statistic:                     2.794
Date:                Fri, 12 May 2023   Prob (F-statistic):           3.43e-17
Time:                        23:01:19   Log-Likelihood:                 1549.7
No. Observations:                 756   AIC:                            -2839.
Df Residuals:                     626   BIC:                            -2238.
Df Model:                         129                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                            0.0383      0.012      3.172      0.002       0.015       0.062
C(raion)[T.Антроповский район]                                      -0.0306      0.023     -1.335      0.182      -0.076       0.014
C(raion)[T.Анучинский район]                                        -0.0174      0.018     -0.963      0.336      -0.053       0.018
C(raion)[T.Арсеньевский]                                            -0.0313      0.023     -1.366      0.173      -0.076       0.014
C(raion)[T.Артемовский]                                             -0.0167      0.018     -0.916      0.360      -0.052       0.019
C(raion)[T.Батецкий район]                                          -0.0311      0.019     -1.627      0.104      -0.069       0.006
C(raion)[T.Бикинский район]                                         -0.0020      0.021     -0.097      0.923      -0.043       0.039
C(raion)[T.Боровичский район]                                       -0.0269      0.018     -1.481      0.139      -0.063       0.009
C(raion)[T.Буйский район]                                           -0.0285      0.018     -1.576      0.115      -0.064       0.007
C(raion)[T.Валдайский район]                                        -0.0195      0.016     -1.211      0.226      -0.051       0.012
C(raion)[T.Ванинский район]                                         -0.0282      0.016     -1.747      0.081      -0.060       0.003
C(raion)[T.Великий Новгород]                                        -0.0310      0.019     -1.669      0.096      -0.068       0.005
C(raion)[T.Верхнебуреинский район]                                  -0.0133      0.019     -0.695      0.487      -0.051       0.024
C(raion)[T.Владивостокский]                                         -0.0294      0.018     -1.616      0.106      -0.065       0.006
C(raion)[T.Волотовский район]                                       -0.0079      0.019     -0.413      0.680      -0.045       0.030
C(raion)[T.Вохомский район]                                         -0.0214      0.019     -1.116      0.265      -0.059       0.016
C(raion)[T.Вяземский район]                                         -0.0211      0.017     -1.270      0.205      -0.054       0.012
C(raion)[T.Галичский район]                                         -0.0343      0.019     -1.791      0.074      -0.072       0.003
C(raion)[T.Город Горно-Алтайск]                                     -0.0248      0.018     -1.348      0.178      -0.061       0.011
C(raion)[T.Город Комсомольск-на-Амуре]                              -0.0120      0.018     -0.652      0.514      -0.048       0.024
C(raion)[T.Город Хабаров

In [40]:
pd.DataFrame(np.array([[1, 2, 3],[4, 5, 6], [7, 8, 9]]))

,0,1,2
0,1,2,3
1,4,5,6
2,7,8,9


In [41]:
table = pd.DataFrame(np.array([roads.params[(len(roads.params)-10):(len(roads.params))].values, 
                               schools.params[(len(schools.params)-10):(len(schools.params))].values, 
                               sewerage.params[(len(sewerage.params)-10):(len(sewerage.params))].values,
                               heat.params[(len(heat.params)-10):(len(heat.params))].values,
                               water.params[(len(water.params)-10):(len(water.params))].values]), columns=["-8", "-7", "-6", "-5", "-4", "-3", "-2", "0", "1", "2"])
table["Зависимая переменная"] = pd.Series(["Качество дорог", "Состояние дошкольных учреждений", "Объем ремонта канализационной сети", "Объем ремонта тепловой сети", "Объем ремонта водопроводной сети"])

In [42]:
table

,-8,-7,-6,-5,-4,-3,-2,0,1,2,Зависимая переменная
0,5.477195,9.923860,6.433814,4.094931,-0.227551,0.922736,2.294867,3.081921,2.839324,5.395110,Качество дорог
1,1.304774,-1.522924,-2.281335,-1.769585,-4.275206,-1.217615,-3.258732,-6.931502,-17.229084,-12.674464,Состояние дошкольных учреждений
2,0.007243,0.006547,0.005055,0.007865,0.011219,0.004543,0.001231,-0.008182,-0.016569,-0.013304,Объем ремонта канализационной сети
3,0.014363,-0.007462,-0.004154,0.001554,-0.008326,0.002099,-0.016125,-0.004555,0.000000,0.000000,Объем ремонта тепловой сети
4,0.024787,-0.001895,0.000006,-0.011050,-0.002130,-0.003245,-0.000615,-0.000771,-0.006981,-0.006880,Объем ремонта водопроводной сети


In [43]:
table.to_excel("D:/DZ/Course_6/Diploma/event_stud_EAS_cor.xlsx")

# Models for financial data

In [44]:
temp_fin.head()

,region,raion,sub_indicator,year,indicator,times,values,budget_type,is_amendment,amendment_date,population
0,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2013,расходы бюджета,Not present,1703.560574,actual_spending,NaN,NaN,271445.0
1,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2014,расходы бюджета,Not present,1757.246786,actual_spending,NaN,NaN,273382.0
2,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2015,расходы бюджета,Not present,1315.128876,actual_spending,NaN,NaN,276090.0
3,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2016,расходы бюджета,Not present,2508.275869,actual_spending,NaN,NaN,276691.0
4,Костромская область,город Кострома,дорожное хозяйство (дорожные фонды),2017,расходы бюджета,Not present,3008.628778,actual_spending,NaN,NaN,277648.0


In [45]:
actual, initial, difference = temp_fin.loc[temp_fin.budget_type == "actual_spending"], temp_fin.loc[temp_fin.budget_type == "initial_budget"], temp_fin.loc[temp_fin.budget_type == "разница"]

In [46]:
actual = pd.pivot(actual, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
initial = pd.pivot(initial, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()
difference = pd.pivot(difference, index = ["region", "raion", "year"], columns="sub_indicator", values="values").reset_index()

In [47]:
for i in actual, initial, difference:
    i.rename(columns={"дорожное хозяйство (дорожные фонды)": "roads", "жилищно-коммунальное хозяйство": "housing", "образование": "education"}, inplace=True)

In [48]:
actual.head()

sub_indicator,region,raion,year,roads,housing,education
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN


In [49]:
frames = [actual, initial, difference]

In [50]:
for j in range(len(frames)):
    #temp_fin_data = pd.DataFrame()
    #for j in mapping.keys():
    #    temp_fin_data = temp_fin_data.append(frames[i].loc[(frames[i].region == j) & ((frames[i].year == mapping[j]) | (frames[i].year == mapping[j] + 5))])
    frames[j]["treatment_period"] = [1 if i > 2018 else 0 for i in  frames[j].year]
    frames[j] = pd.merge(frames[j], data_for_models, on=["region", "raion"], how="inner")

In [51]:
for j in range(len(frames)): 
    for i in range(-8,4):
        if i < 0:
            string = f"minus_{abs(i)}"
        else:
            string = f'plus_{i}'
        frames[j][string] = 0
        for region in mapping.keys():
            frames[j].loc[(frames[j].region == region) & (frames[j].year == mapping[region]+5+i) & (frames[j].treatment_1 == 1), string] = 1

In [52]:
frames[0].loc[frames[0].treatment_1 == 1]

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
50,Костромская область,Костромской район,2013,361.610727,684.797466,8396.563320,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0
51,Костромская область,Костромской район,2014,313.010857,976.450532,9238.023379,0,1,1,0,0,1,0,0,0,0,0,0,0,0,0
52,Костромская область,Костромской район,2015,190.454672,479.029493,17641.915178,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0
53,Костромская область,Костромской район,2016,753.596638,2185.095724,7938.800846,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0
54,Костромская область,Костромской район,2017,NaN,NaN,NaN,0,1,1,0,0,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1415,Хабаровский край,Город Хабаровск,2018,2366.283882,1482.057687,9182.517891,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0
1416,Хабаровский край,Город Хабаровск,2019,2333.307617,1014.289413,9728.819074,1,1,1,0,0,0,0,0,0,0,0,1,0,0,0
1417,Хабаровский край,Город Хабаровск,2020,2418.134198,976.163275,9177.532782,1,1,1,0,0,0,0,0,0,0,0,0,1,0,0
1418,Хабаровский край,Город Хабаровск,2021,2054.200464,1327.566750,9218.851314,1,1,1,0,0,0,0,0,0,0,0,0,0,1,0


In [53]:
frames[2].year.max()

2022

In [54]:
#frames[0]

In [55]:
actual, initial, difference = frames

In [56]:
actual.head()

,region,raion,year,roads,housing,education,treatment_period,treatment_1,treatment_2,minus_8,minus_7,minus_6,minus_5,minus_4,minus_3,minus_2,minus_1,plus_0,plus_1,plus_2,plus_3
0,Костромская область,Антроповский район,2013,666.115702,1610.368144,12882.945154,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
1,Костромская область,Антроповский район,2014,603.232937,1922.796250,12638.114480,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
2,Костромская область,Антроповский район,2015,688.908710,95.255228,21901.154847,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,Костромская область,Антроповский район,2016,764.043811,203.425719,10986.123190,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,Костромская область,Антроповский район,2017,NaN,NaN,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


In [57]:
actual_roads_data = actual[["raion", "region", "roads", "year", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2","minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_roads_data.dropna(inplace=True)
actual_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_roads_data).fit()
actual_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.466
Model:                            OLS   Adj. R-squared:                  0.374
Method:                 Least Squares   F-statistic:                     5.075
Date:                Fri, 12 May 2023   Prob (F-statistic):           9.40e-60
Time:                        23:01:46   Log-Likelihood:                -9485.2
No. Observations:                1147   AIC:                         1.931e+04
Df Residuals:                     978   BIC:                         2.016e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          348.5339    354.267      0.984      0.325    -346.676    1043.744
C(raion)[T.Антроповский район]                                     544.8903    571.352      0.954      0.340    -576.327    1666.107
C(raion)[T.Анучинский район]                                       762.1053    515.848      1.477      0.140    -250.190    1774.401
C(raion)[T.Арсеньевский]                                           512.3630    482.188      1.063      0.288    -433.879    1458.605
C(raion)[T.Артемовский]                                           1265.8998    516.680      2.450      0.014     251.972    2279.828
C(raion)[T.Аяно-Майский район]                                    2560.3153    482.188      5.310      0.000    1614.073    3506.558
C(raion)[T.Батецкий район]                                        -247.2697    482.188     -0.513      0.608   -1193.512     698.973
C(raion)[T.Бикинский район]                                       -320.8584    482.188     -0.665      0.506   -1267.101     625.384
C(raion)[T.Боровичский район]                                      344.4130    516.680      0.667      0.505    -669.515    1358.341
C(raion)[T.Буйский район]                                          867.5021    571.352      1.518      0.129    -253.715    1988.719
C(raion)[T.Валдайский район]                                       169.6356    482.188      0.352      0.725    -776.607    1115.878
C(raion)[T.Ванинский район]                                        -81.9262    482.188     -0.170      0.865   -1028.169     864.316
C(raion)[T.Великий Новгород]                                      1646.6328    516.680      3.187      0.001     632.705    2660.561
C(raion)[T.Верхнебуреинский район]                                 732.7835    482.188      1.520      0.129    -213.459    1679.026
C(raion)[T.Владивостокский]                                       2415.1002    516.680      4.674      0.000    1401.172    3429.028
C(raion)[T.Волжский район]                                         241.8019    529.968      0.456      0.648    -798.203    1281.807
C(raion)[T.Волотовский район]                                     -164.1712    497.209     -0.330      0.741   -1139.890     811.547
C(raion)[T.Вохомский район]                                       2127.9017    571.367      3.724      0.000    1006.655    3249.148
C(raion)[T.Вяземский район]                                        121.2266    482.188      0.251      0.802    -825.016    1067.469
C(raion)[T.Галичский район]                                        843.9858    539.626      1.564      0.118    -214.973    1902.945
C(raion)[T.Горномарийски

In [58]:
actual_housing_data = actual[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_housing_data.dropna(inplace=True)
actual_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_housing_data).fit()
actual_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.884
Model:                            OLS   Adj. R-squared:                  0.864
Method:                 Least Squares   F-statistic:                     45.06
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:01:47   Log-Likelihood:                -11274.
No. Observations:                1163   AIC:                         2.289e+04
Df Residuals:                     994   BIC:                         2.374e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         2157.1586   1466.387      1.471      0.142    -720.410    5034.728
C(raion)[T.Антроповский район]                                    -350.1056   2370.314     -0.148      0.883   -5001.499    4301.288
C(raion)[T.Анучинский район]                                     -1114.4669   2140.085     -0.521      0.603   -5314.069    3085.135
C(raion)[T.Арсеньевский]                                           799.8042   2000.463      0.400      0.689   -3125.812    4725.421
C(raion)[T.Артемовский]                                            389.5806   2138.561      0.182      0.855   -3807.032    4586.194
C(raion)[T.Аяно-Майский район]                                    5.699e+04   2000.463     28.490      0.000    5.31e+04    6.09e+04
C(raion)[T.Батецкий район]                                        -442.0961   2000.463     -0.221      0.825   -4367.713    3483.520
C(raion)[T.Бикинский район]                                       1084.5129   2000.463      0.542      0.588   -2841.104    5010.129
C(raion)[T.Боровичский район]                                     -955.1579   2138.561     -0.447      0.655   -5151.771    3241.455
C(raion)[T.Буйский район]                                        -1352.2891   2554.754     -0.529      0.597   -6365.620    3661.042
C(raion)[T.Валдайский район]                                      -750.3068   2000.463     -0.375      0.708   -4675.923    3175.310
C(raion)[T.Ванинский район]                                       9769.9337   2000.463      4.884      0.000    5844.317    1.37e+04
C(raion)[T.Великий Новгород]                                       455.2020   2138.561      0.213      0.831   -3741.411    4651.815
C(raion)[T.Верхнебуреинский район]                                5032.4429   2000.463      2.516      0.012    1106.827    8958.059
C(raion)[T.Владивостокский]                                        285.6466   2138.561      0.134      0.894   -3910.966    4482.260
C(raion)[T.Волжский район]                                       -1338.3316   2195.760     -0.610      0.542   -5647.188    2970.525
C(raion)[T.Волотовский район]                                      867.4878   2062.771      0.421      0.674   -3180.398    4915.373
C(raion)[T.Вохомский район]                                        414.8201   2370.359      0.175      0.861   -4236.662    5066.302
C(raion)[T.Вяземский район]                                       2856.1783   2000.463      1.428      0.154   -1069.438    6781.795
C(raion)[T.Галичский район]                                      -1143.2188   2238.722     -0.511      0.610   -5536.382    3249.944
C(raion)[T.Горномарийски

In [59]:
actual_education_data = actual[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
actual_education_data.dropna(inplace=True)
actual_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=actual_education_data).fit()
actual_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.899
Model:                            OLS   Adj. R-squared:                  0.882
Method:                 Least Squares   F-statistic:                     53.53
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:01:48   Log-Likelihood:                -11052.
No. Observations:                1182   AIC:                         2.244e+04
Df Residuals:                    1013   BIC:                         2.330e+04
Df Model:                         168                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.789e+04   1037.192     17.248      0.000    1.59e+04    1.99e+04
C(raion)[T.Антроповский район]                                   -1891.8872   1678.453     -1.127      0.260   -5185.531    1401.756
C(raion)[T.Анучинский район]                                     -3869.7748   1515.459     -2.554      0.011   -6843.573    -895.976
C(raion)[T.Арсеньевский]                                         -6579.0110   1416.589     -4.644      0.000   -9358.796   -3799.225
C(raion)[T.Артемовский]                                          -6443.0792   1514.211     -4.255      0.000   -9414.429   -3471.729
C(raion)[T.Аяно-Майский район]                                    4.413e+04   1416.589     31.153      0.000    4.14e+04    4.69e+04
C(raion)[T.Батецкий район]                                       -4923.0793   1416.589     -3.475      0.001   -7702.865   -2143.294
C(raion)[T.Бикинский район]                                       -657.5799   1416.589     -0.464      0.643   -3437.365    2122.206
C(raion)[T.Боровичский район]                                    -7296.7278   1514.211     -4.819      0.000   -1.03e+04   -4325.378
C(raion)[T.Буйский район]                                        -3123.4542   1678.453     -1.861      0.063   -6417.098     170.189
C(raion)[T.Валдайский район]                                     -7065.9048   1416.589     -4.988      0.000   -9845.690   -4286.119
C(raion)[T.Ванинский район]                                        504.7007   1416.589      0.356      0.722   -2275.085    3284.486
C(raion)[T.Великий Новгород]                                     -7753.8377   1514.211     -5.121      0.000   -1.07e+04   -4782.488
C(raion)[T.Верхнебуреинский район]                                6874.1566   1416.589      4.853      0.000    4094.371    9653.942
C(raion)[T.Владивостокский]                                      -7916.8178   1514.211     -5.228      0.000   -1.09e+04   -4945.468
C(raion)[T.Волжский район]                                       -6266.0153   1554.806     -4.030      0.000   -9317.024   -3215.006
C(raion)[T.Волотовский район]                                    -7145.2461   1460.716     -4.892      0.000      -1e+04   -4278.870
C(raion)[T.Вохомский район]                                        349.9212   1678.498      0.208      0.835   -2943.810    3643.652
C(raion)[T.Вяземский район]                                       3552.5245   1416.589      2.508      0.012     772.739    6332.310
C(raion)[T.Галичский район]                                      -3948.0366   1585.287     -2.490      0.013   -7058.860    -837.214
C(raion)[T.Горномарийски

In [60]:
initial_roads_data = initial[["raion", "region", "year", "roads", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_roads_data.dropna(inplace=True)
initial_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_roads_data).fit()
initial_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.307
Model:                            OLS   Adj. R-squared:                  0.174
Method:                 Least Squares   F-statistic:                     2.313
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.93e-13
Time:                        23:01:48   Log-Likelihood:                -7861.9
No. Observations:                 911   AIC:                         1.602e+04
Df Residuals:                     764   BIC:                         1.673e+04
Df Model:                         146                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                           99.2810    556.454      0.178      0.858    -993.080    1191.641
C(raion)[T.Антроповский район]                                     675.1916    739.417      0.913      0.361    -776.339    2126.722
C(raion)[T.Анучинский район]                                       812.1747    844.259      0.962      0.336    -845.167    2469.517
C(raion)[T.Арсеньевский]                                           161.1737    739.417      0.218      0.828   -1290.357    1612.705
C(raion)[T.Артемовский]                                            610.7458    784.904      0.778      0.437    -930.079    2151.571
C(raion)[T.Аяно-Майский район]                                    2700.7786    739.417      3.653      0.000    1249.248    4152.309
C(raion)[T.Батецкий район]                                        -160.7319    765.853     -0.210      0.834   -1664.158    1342.694
C(raion)[T.Бикинский район]                                       -132.7269    739.417     -0.180      0.858   -1584.258    1318.804
C(raion)[T.Боровичский район]                                      279.1777    807.203      0.346      0.730   -1305.422    1863.778
C(raion)[T.Буйский район]                                          488.4342    765.853      0.638      0.524   -1014.992    1991.860
C(raion)[T.Валдайский район]                                       188.7024    765.853      0.246      0.805   -1314.724    1692.129
C(raion)[T.Ванинский район]                                        -26.5885    739.417     -0.036      0.971   -1478.119    1424.942
C(raion)[T.Великий Новгород]                                       -96.7463    784.904     -0.123      0.902   -1637.571    1444.079
C(raion)[T.Верхнебуреинский район]                                -215.3413    739.417     -0.291      0.771   -1666.872    1236.190
C(raion)[T.Владивостокский]                                       1388.9773    784.904      1.770      0.077    -151.848    2929.802
C(raion)[T.Волотовский район]                                     -167.7339    765.660     -0.219      0.827   -1670.781    1335.313
C(raion)[T.Вохомский район]                                        736.2334    765.853      0.961      0.337    -767.193    2239.660
C(raion)[T.Вяземский район]                                        109.7405    739.417      0.148      0.882   -1341.790    1561.271
C(raion)[T.Галичский район]                                        745.6439    765.853      0.974      0.331    -757.782    2249.070
C(raion)[T.Город Горно-Алтайск]                                   1417.9569    966.210      1.468      0.143    -478.784    3314.698
C(raion)[T.Город Комсомо

In [61]:
initial_housing_data = initial[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_housing_data.dropna(inplace=True)
initial_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_housing_data).fit()
initial_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.750
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     15.91
Date:                Fri, 12 May 2023   Prob (F-statistic):          1.05e-158
Time:                        23:01:48   Log-Likelihood:                -9363.2
No. Observations:                 929   AIC:                         1.902e+04
Df Residuals:                     781   BIC:                         1.974e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          408.2995   2335.844      0.175      0.861   -4176.977    4993.576
C(raion)[T.Антроповский район]                                   -1132.9916   3859.381     -0.294      0.769   -8708.981    6442.997
C(raion)[T.Анучинский район]                                      -441.4341   3590.246     -0.123      0.902   -7489.110    6606.241
C(raion)[T.Арсеньевский]                                           928.4250   3144.753      0.295      0.768   -5244.744    7101.594
C(raion)[T.Артемовский]                                           -546.6396   3330.944     -0.164      0.870   -7085.304    5992.025
C(raion)[T.Аяно-Майский район]                                    4.955e+04   3144.753     15.757      0.000    4.34e+04    5.57e+04
C(raion)[T.Батецкий район]                                        -973.6982   3256.731     -0.299      0.765   -7366.682    5419.285
C(raion)[T.Бикинский район]                                       1277.5350   3144.753      0.406      0.685   -4895.634    7450.704
C(raion)[T.Боровичский район]                                    -1148.0085   3428.232     -0.335      0.738   -7877.649    5581.632
C(raion)[T.Буйский район]                                        -1000.6170   3256.731     -0.307      0.759   -7393.600    5392.366
C(raion)[T.Валдайский район]                                      -974.0031   3400.011     -0.286      0.775   -7648.246    5700.240
C(raion)[T.Ванинский район]                                       9064.7576   3144.753      2.883      0.004    2891.589    1.52e+04
C(raion)[T.Великий Новгород]                                     -1689.7970   3330.944     -0.507      0.612   -8228.461    4848.867
C(raion)[T.Верхнебуреинский район]                                -957.1933   3144.753     -0.304      0.761   -7130.362    5215.976
C(raion)[T.Владивостокский]                                       -140.9340   3330.944     -0.042      0.966   -6679.598    6397.730
C(raion)[T.Волотовский район]                                     -992.1082   3256.345     -0.305      0.761   -7384.332    5400.116
C(raion)[T.Вохомский район]                                       -798.6678   3256.731     -0.245      0.806   -7191.651    5594.316
C(raion)[T.Вяземский район]                                       2499.8620   3144.753      0.795      0.427   -3673.307    8673.031
C(raion)[T.Галичский район]                                       -840.0690   3256.731     -0.258      0.797   -7233.052    5552.914
C(raion)[T.Город Горно-Алтайск]                                    869.2052   4103.406      0.212      0.832   -7185.805    8924.215
C(raion)[T.Город Комсомо

In [62]:
initial_education_data = initial[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
initial_education_data.dropna(inplace=True)
initial_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=initial_education_data).fit()
initial_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.951
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     105.3
Date:                Fri, 12 May 2023   Prob (F-statistic):               0.00
Time:                        23:01:49   Log-Likelihood:                -8527.1
No. Observations:                 953   AIC:                         1.735e+04
Df Residuals:                     805   BIC:                         1.807e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1.609e+04    751.010     21.426      0.000    1.46e+04    1.76e+04
C(raion)[T.Антроповский район]                                   -6153.2268   1012.283     -6.079      0.000   -8140.252   -4166.201
C(raion)[T.Анучинский район]                                     -2226.9388   1155.637     -1.927      0.054   -4495.355      41.478
C(raion)[T.Арсеньевский]                                         -5571.2562   1012.283     -5.504      0.000   -7558.282   -3584.231
C(raion)[T.Артемовский]                                          -5778.4942   1072.012     -5.390      0.000   -7882.763   -3674.225
C(raion)[T.Аяно-Майский район]                                    3.437e+04   1012.283     33.954      0.000    3.24e+04    3.64e+04
C(raion)[T.Батецкий район]                                       -4314.1778   1048.315     -4.115      0.000   -6371.932   -2256.424
C(raion)[T.Бикинский район]                                      -1732.6383   1012.283     -1.712      0.087   -3719.664     254.387
C(raion)[T.Боровичский район]                                    -6434.6642   1103.075     -5.833      0.000   -8599.906   -4269.422
C(raion)[T.Буйский район]                                        -4840.8661   1048.315     -4.618      0.000   -6898.620   -2783.112
C(raion)[T.Валдайский район]                                     -6598.0366   1048.315     -6.294      0.000   -8655.790   -4540.283
C(raion)[T.Ванинский район]                                       2182.6928   1012.283      2.156      0.031     195.667    4169.718
C(raion)[T.Великий Новгород]                                     -1.436e+04   1072.012    -13.399      0.000   -1.65e+04   -1.23e+04
C(raion)[T.Верхнебуреинский район]                               -1.513e+04   1012.283    -14.942      0.000   -1.71e+04   -1.31e+04
C(raion)[T.Владивостокский]                                      -6675.6159   1072.012     -6.227      0.000   -8779.885   -4571.347
C(raion)[T.Волотовский район]                                    -1.502e+04   1048.204    -14.327      0.000   -1.71e+04    -1.3e+04
C(raion)[T.Вохомский район]                                      -3116.0062   1048.315     -2.972      0.003   -5173.760   -1058.252
C(raion)[T.Вяземский район]                                       4639.9189   1012.283      4.584      0.000    2652.894    6626.944
C(raion)[T.Галичский район]                                      -4996.4269   1048.315     -4.766      0.000   -7054.181   -2938.673
C(raion)[T.Город Горно-Алтайск]                                  -2265.8608   1320.820     -1.715      0.087   -4858.518     326.796
C(raion)[T.Город Комсомо

In [63]:
difference_roads_data = difference[["raion", "region", "year", "roads", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_roads_data.dropna(inplace=True)
difference_roads = sm_f.ols(formula="roads ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_roads_data).fit()
difference_roads.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  roads   R-squared:                       0.372
Model:                            OLS   Adj. R-squared:                  0.232
Method:                 Least Squares   F-statistic:                     2.663
Date:                Fri, 12 May 2023   Prob (F-statistic):           4.89e-17
Time:                        23:01:49   Log-Likelihood:                -6451.7
No. Observations:                 798   AIC:                         1.320e+04
Df Residuals:                     652   BIC:                         1.388e+04
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          193.3906    327.481      0.591      0.555    -449.654     836.435
C(raion)[T.Антроповский район]                                     187.7905    533.134      0.352      0.725    -859.077    1234.658
C(raion)[T.Анучинский район]                                       415.3809    496.013      0.837      0.403    -558.595    1389.357
C(raion)[T.Арсеньевский]                                           423.2823    434.329      0.975      0.330    -429.571    1276.135
C(raion)[T.Артемовский]                                            977.1414    467.327      2.091      0.037      59.493    1894.790
C(raion)[T.Аяно-Майский район]                                       6.9539    434.329      0.016      0.987    -845.899     859.807
C(raion)[T.Батецкий район]                                        -141.8866    449.867     -0.315      0.753   -1025.249     741.476
C(raion)[T.Бикинский район]                                       -142.2890    434.329     -0.328      0.743    -995.142     710.564
C(raion)[T.Боровичский район]                                      144.6373    480.136      0.301      0.763    -798.163    1087.437
C(raion)[T.Буйский район]                                          105.5191    589.657      0.179      0.858   -1052.336    1263.374
C(raion)[T.Валдайский район]                                      -131.9968    449.867     -0.293      0.769   -1015.359     751.366
C(raion)[T.Ванинский район]                                         26.8521    434.329      0.062      0.951    -826.001     879.705
C(raion)[T.Великий Новгород]                                      1995.0845    467.327      4.269      0.000    1077.436    2912.733
C(raion)[T.Верхнебуреинский район]                                1079.4391    434.329      2.485      0.013     226.586    1932.292
C(raion)[T.Владивостокский]                                       1324.8765    467.327      2.835      0.005     407.228    2242.525
C(raion)[T.Волотовский район]                                      -13.2098    449.779     -0.029      0.977    -896.401     869.981
C(raion)[T.Вохомский район]                                       2352.3635    589.649      3.989      0.000    1194.524    3510.203
C(raion)[T.Вяземский район]                                        -52.6785    434.329     -0.121      0.904    -905.531     800.174
C(raion)[T.Галичский район]                                        596.9050    533.034      1.120      0.263    -449.765    1643.576
C(raion)[T.Город Горно-Алтайск]                                   2044.6457    578.612      3.534      0.000     908.479    3180.813
C(raion)[T.Город Комсомо

In [64]:
difference_housing_data = difference[["raion", "region", "year", "housing", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_housing_data.dropna(inplace=True)
difference_housing = sm_f.ols(formula="housing ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_housing_data).fit()
difference_housing.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                housing   R-squared:                       0.353
Model:                            OLS   Adj. R-squared:                  0.213
Method:                 Least Squares   F-statistic:                     2.523
Date:                Fri, 12 May 2023   Prob (F-statistic):           2.12e-15
Time:                        23:01:50   Log-Likelihood:                -7952.9
No. Observations:                 816   AIC:                         1.620e+04
Df Residuals:                     670   BIC:                         1.688e+04
Df Model:                         145                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                         1618.1367   1700.857      0.951      0.342   -1721.515    4957.788
C(raion)[T.Антроповский район]                                     749.9797   4865.657      0.154      0.878   -8803.791    1.03e+04
C(raion)[T.Анучинский район]                                      -575.1337   2605.452     -0.221      0.825   -5690.968    4540.701
C(raion)[T.Арсеньевский]                                          -158.4673   2281.642     -0.069      0.945   -4638.496    4321.562
C(raion)[T.Артемовский]                                            197.7220   2447.124      0.081      0.936   -4607.232    5002.676
C(raion)[T.Аяно-Майский район]                                    8276.4726   2281.642      3.627      0.000    3796.444    1.28e+04
C(raion)[T.Батецкий район]                                        -134.8405   2362.976     -0.057      0.955   -4774.570    4504.889
C(raion)[T.Бикинский район]                                       -240.9125   2281.642     -0.106      0.916   -4720.941    4239.116
C(raion)[T.Боровичский район]                                     -948.4852   2516.451     -0.377      0.706   -5889.564    3992.594
C(raion)[T.Буйский район]                                         -230.7088   3619.962     -0.064      0.949   -7338.544    6877.126
C(raion)[T.Валдайский район]                                      -588.9624   2467.057     -0.239      0.811   -5433.056    4255.131
C(raion)[T.Ванинский район]                                        373.6769   2281.642      0.164      0.870   -4106.352    4853.706
C(raion)[T.Великий Новгород]                                      1457.4879   2447.124      0.596      0.552   -3347.466    6262.442
C(raion)[T.Верхнебуреинский район]                                5891.3618   2281.642      2.582      0.010    1411.333    1.04e+04
C(raion)[T.Владивостокский]                                       -207.3736   2447.124     -0.085      0.932   -5012.328    4597.581
C(raion)[T.Волотовский район]                                     1911.9859   2362.795      0.809      0.419   -2727.389    6551.361
C(raion)[T.Вохомский район]                                       2205.9863   3097.363      0.712      0.477   -3875.721    8287.693
C(raion)[T.Вяземский район]                                        203.1508   2281.642      0.089      0.929   -4276.878    4683.180
C(raion)[T.Галичский район]                                        136.2639   2799.869      0.049      0.961   -5361.309    5633.837
C(raion)[T.Город Горно-Алтайск]                                   -177.8909   3031.182     -0.059      0.953   -6129.650    5773.869
C(raion)[T.Город Комсомо

In [65]:
difference_education_data = difference[["raion", "region", "year", "education", "minus_8", "minus_7", "minus_6", "minus_5", "minus_4", "minus_3", "minus_2", "minus_1", "plus_0", "plus_1", "plus_2", "plus_3"]].copy()
difference_education_data.dropna(inplace=True)
difference_education = sm_f.ols(formula="education ~ C(raion) + C(year) + minus_8 + minus_7 + minus_6 + minus_5 + minus_4 + minus_3 + minus_1 + plus_0 + plus_1 + plus_2", data=difference_education_data).fit()
difference_education.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              education   R-squared:                       0.613
Model:                            OLS   Adj. R-squared:                  0.530
Method:                 Least Squares   F-statistic:                     7.377
Date:                Fri, 12 May 2023   Prob (F-statistic):           4.38e-75
Time:                        23:01:50   Log-Likelihood:                -7773.3
No. Observations:                 834   AIC:                         1.584e+04
Df Residuals:                     686   BIC:                         1.654e+04
Df Model:                         147                                         
Covariance Type:            nonrobust                                         
====================================================================================================================================
                                                                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------------------------------
Intercept                                                          806.4436   1107.800      0.728      0.467   -1368.641    2981.529
C(raion)[T.Антроповский район]                                    5084.1665   1827.945      2.781      0.006    1495.128    8673.205
C(raion)[T.Анучинский район]                                     -1528.8855   1700.691     -0.899      0.369   -4868.070    1810.299
C(raion)[T.Арсеньевский]                                         -1085.3737   1489.409     -0.729      0.466   -4009.721    1838.973
C(raion)[T.Артемовский]                                           -668.4534   1597.797     -0.418      0.676   -3805.614    2468.707
C(raion)[T.Аяно-Майский район]                                    8617.7915   1489.409      5.786      0.000    5693.445    1.15e+04
C(raion)[T.Батецкий район]                                        -174.3985   1542.466     -0.113      0.910   -3202.919    2854.122
C(raion)[T.Бикинский район]                                        492.5956   1489.409      0.331      0.741   -2431.751    3416.942
C(raion)[T.Боровичский район]                                     -845.4966   1642.756     -0.515      0.607   -4070.930    2379.937
C(raion)[T.Буйский район]                                         4432.4082   2021.780      2.192      0.029     462.788    8402.029
C(raion)[T.Валдайский район]                                      -233.9758   1542.466     -0.152      0.879   -3262.496    2794.544
C(raion)[T.Ванинский район]                                      -2133.6686   1489.409     -1.433      0.152   -5058.015     790.678
C(raion)[T.Великий Новгород]                                      6547.1497   1597.797      4.098      0.000    3409.990    9684.310
C(raion)[T.Верхнебуреинский район]                                2.198e+04   1489.409     14.758      0.000    1.91e+04    2.49e+04
C(raion)[T.Владивостокский]                                      -1330.3111   1597.797     -0.833      0.405   -4467.471    1806.849
C(raion)[T.Волотовский район]                                     7608.4877   1542.377      4.933      0.000    4580.141    1.06e+04
C(raion)[T.Вохомский район]                                       2899.1129   2021.723      1.434      0.152   -1070.394    6868.620
C(raion)[T.Вяземский район]                                      -1247.9966   1489.409     -0.838      0.402   -4172.343    1676.350
C(raion)[T.Галичский район]                                       2875.1961   1827.628      1.573      0.116    -713.220    6463.612
C(raion)[T.Город Горно-Алтайск]                                    647.2195   1978.635      0.327      0.744   -3237.688    4532.127
C(raion)[T.Город Комсомо

In [66]:
table = pd.DataFrame(np.array([actual_roads.params[(len(actual_roads.params)-10):(len(actual_roads.params))].values, 
                               actual_housing.params[(len(actual_housing.params)-10):(len(actual_housing.params))].values, 
                               actual_education.params[(len(actual_education.params)-10):(len(actual_education.params))].values,
                               initial_roads.params[(len(initial_roads.params)-10):(len(initial_roads.params))].values, 
                               initial_housing.params[(len(initial_housing.params)-10):(len(initial_housing.params))].values, 
                               initial_education.params[(len(initial_education.params)-10):(len(initial_education.params))].values,
                               difference_roads.params[(len(difference_roads.params)-10):(len(difference_roads.params))].values, 
                               difference_housing.params[(len(difference_housing.params)-10):(len(difference_housing.params))].values, 
                               difference_education.params[(len(difference_education.params)-10):(len(difference_education.params))].values]), columns=["-8", "-7", "-6", "-5", "-4", "-3", "-2", "0", "1", "2"])
table["Зависимая переменная"] = pd.Series(["Дорожное хозяйство", "ЖКХ", "Образование", "Дорожное хозяйство", "ЖКХ", "Образование", "Дорожное хозяйство", "ЖКХ", "Образование"])
table["Тип расходов"] = pd.Series(["Фактические", "Фактические", "Фактические", "Запланированные", "Запланированные", "Запланированные", "Фактические - запланированные", "Фактические - запланированные", "Фактические - запланированные"])

In [67]:
table

,-8,-7,-6,-5,-4,-3,-2,0,1,2,Зависимая переменная,Тип расходов
0,-3.754301e+02,-552.811922,-580.574785,-300.719601,-224.983535,-382.732205,-40.616075,-104.794031,-51.628782,-1.336412,Дорожное хозяйство,Фактические
1,7.221986e+02,626.382457,868.940520,753.682325,543.401670,381.001092,255.704802,47.077407,316.857545,130.493326,ЖКХ,Фактические
2,-2.138479e+03,-1216.238532,-1166.363712,1220.223783,-439.975805,370.105890,281.121842,433.841134,133.221102,1868.505133,Образование,Фактические
3,2.989384e-13,-172.621297,-132.633728,-3.895038,-166.136351,-219.856475,-45.130593,-203.209514,10.372872,-369.679798,Дорожное хозяйство,Запланированные
4,-3.383294e-12,2793.578584,561.898070,939.222246,559.511670,761.063054,-3.358564,236.783965,21.657645,613.713742,ЖКХ,Запланированные
5,-6.018621e-13,-2582.155985,-1738.426939,-952.616151,-584.815720,-414.824202,511.530950,-428.167968,-1039.983691,44.193291,Образование,Запланированные
6,-4.161382e-13,-595.197242,-670.927753,-589.280677,-289.683560,-399.977078,-159.340758,-231.270553,-183.606483,182.357971,Дорожное хозяйство,Фактические - запланированные
7,-1.108865e-12,-1045.652721,932.428712,607.165954,665.285979,203.337400,941.985611,624.705159,897.393329,572.898768,ЖКХ,Фактические - запланированные
8,-2.378034e-13,1996.739797,166.777400,1695.062669,-87.338874,656.209243,-581.658212,660.745780,1206.747947,1492.815242,Образование,Фактические - запланированные


In [68]:
table.to_excel("D:/DZ/Course_6/Diploma/event_stud_fin_EAS_cor.xlsx")